# GraphSL-based Root Prediction via Graph Convolution

This notebook adapts recent **graph source localization** methods from the GraphSL library to our root-prediction task on syntactic dependency trees.  
Graph source localization is the inverse of graph diffusion: the main goal is to identify the origin(s) of an observed cascade of “infections” on a network. In our setting, the tree’s **root** plays the role of the diffusion source, and the dependency edges define how an infection might propagate outward. 

GraphSL (“Graph Source Localization”) was introduced by Wang & Zhao et al. (2024) to provide a unified framework for simulating graph diffusion and benchmarking localization algorithms.

> **Why GraphSL?**  
> - **Inversion of diffusion** matches our root‐prediction task: roots “infect” dependents.  
> - **Modular implementations** of LPSI, NetSleuth, OJC let us compare distinct heuristics.  
> - **Future potential**: with more compute and hyperparameter tuning, a GraphSL-based approach could exceed tree‐centric XGBoost models, as supported by the literature used to create this library.

**GraphSL: An Open-Source Library for Graph Source Localization Approaches and Benchmark Datasets**, Wang & Zhao et al. (2024).

The [**repository**](https://github.com/xianggebenben/GraphSL) and the [**arXiv article**](https://arxiv.org/html/2405.03724v1#:~:text=extremely%20important%20topic%3A%20it%20focuses,investigations%20from%20machine%20learning%20researchers) are both publicly available.

If we had more time to work on the project, this approach could have been perfected a lot better with some modifications (even becoming a main non-linear model), but in its current state, it can only stay as an experiment.

To see why the necessary data transformations were performed, look at the benchmarking dataset formats in the repository.

In [ ]:
# !pip install GraphSL

In [2]:
import io
import contextlib
import ast
import pandas as pd
import networkx as nx
import numpy as np
import torch
from scipy import sparse
from GraphSL.utils import diffusion_generation, split_dataset
from GraphSL.Prescribed import LPSI, NetSleuth, OJC
import warnings
warnings.filterwarnings("ignore")

In [9]:
# parameters, these could also be optimized in future work
INFECT_PROB   = 0.3
SEED_RATIO    = 0.2
SIM_NUM_TRAIN = 100
TRAIN_RATIO   = 0.8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", default_device)

Using device: cuda


In [4]:
train_df = pd.read_csv("../../data/train.csv")
test_df  = pd.read_csv("../../data/test.csv")

In [5]:
# only keep Polish for the test as this ensable runs for quite long
train_df = train_df[train_df.language == "Polish"].reset_index(drop=True)
test_df  = test_df[test_df.language  == "Polish"].reset_index(drop=True)

In [6]:
# build graphs
def build_graph(edgelist_str, n):
    edges = ast.literal_eval(edgelist_str)
    G = nx.Graph()
    G.add_nodes_from(range(1, n+1))
    G.add_edges_from(edges)
    return G

train_df["G"] = train_df.apply(lambda r: build_graph(r["edgelist"], r["n"]), axis=1)
test_df ["G"] = test_df .apply(lambda r: build_graph(r["edgelist"], r["n"]), axis=1)

We generate IC cascades on each graph to create “diffusion observations,” then split into train/val sets for hyperparameter tuning.

In [7]:
# generate and split cascades for every Polish graph
splits = []
for G in train_df["G"]:
    A_sp = nx.to_scipy_sparse_array(G, dtype=float, format="csr")
    data = diffusion_generation(
        graph       = {"adj_mat": A_sp},
        diff_type   = "IC",
        sim_num     = SIM_NUM_TRAIN,
        seed_ratio  = SEED_RATIO,
        infect_prob = INFECT_PROB
    )
    splits.append(split_dataset(data, train_ratio=TRAIN_RATIO))
print(f"Prepared {len(splits)} train/val splits on Polish graphs")

Prepared 500 train/val splits on Polish graphs


### GraphSL Methods & Tuning
GraphSL has three prescribed source localization algorithms:
- LPSI: Labels local peaks in a diffusion score vector as sources.
- NetSleuth: Minimum description length heuristic to cover infected nodes.
- OJC: Jordan cover selects nodes minimizing maximum distance to observed infected nodes.

We simulate cascades on one representative graph (the first sentence per language), tune each method on 100 IC cascades, and pick the best by validation AUC.

> **LPSI tuning**: we sweep α over $$logspace(-4,-1)\$$ preferring small α to avoid over‐smoothing.

In [10]:
# tune LPSI by averaging val-AUC over all splits
print("\nTuning LPSI…")
alpha_grid, best_alpha, best_auc = np.logspace(-4, -1, 10), None, -1.0
for α in alpha_grid:
    aucs = []
    for adj_mat, train_ds, val_ds in splits:
        m = LPSI()
        m.device = DEVICE
        # silence its prints, only included for OJC,
        with contextlib.redirect_stdout(io.StringIO()):
            _, _, auc, _, _ = m.train(adj_mat, train_ds,
                                      alpha_list=[α], num_thres=10)
        aucs.append(auc)
    mean_auc = np.mean(aucs)
    if mean_auc > best_auc:
        best_auc, best_alpha = mean_auc, α
print(f" -> best α={best_alpha:.4f}, mean val-AUC={best_auc:.3f}")


Tuning LPSI…
 → best α=0.0010, mean val-AUC=0.652


> **NetSleuth tuning**: we test β ∈ {2,5,10} on the same splits.

In [11]:
# tune NetSleuth by averaging val-AUC over all splits
print("\nTuning NetSleuth…")
beta_grid = [2,5,10]  # NetSleuth only supports a single β per split_train
best_beta, best_auc_n = None, -1.0
for β in beta_grid:
    aucs = []
    for adj_mat, train_ds, val_ds in splits:
        m = NetSleuth()
        m.device = DEVICE
        with contextlib.redirect_stdout(io.StringIO()):
            _, auc, _ = m.train(adj_mat, train_ds)
        aucs.append(auc)
    mean_auc = np.mean(aucs)
    if mean_auc > best_auc_n:
        best_auc_n, best_beta = mean_auc, β
print(f" -> best β={best_beta:.3f}, mean val-AUC={best_auc_n:.3f}")


Tuning NetSleuth…
 → best β=2.000, mean val-AUC=0.536


> **OJC tuning**: test small K values; note API differences between versions.

In [19]:
# tune OJC by averaging val‐AUC over all splits
print("\nTuning OJC…")
K_grid = [1, 2, 3] # the few K values you want to try
best_K = None
best_auc_o = -1.0
for K in K_grid:
    aucs = []
    for adj_mat, train_ds, val_ds in splits:
        m = OJC()
        # try the new signature
        try:
            res = m.train(adj_mat, train_ds, num_source=K)
        except TypeError:
            # fallback to the old signature
            res = m.train(adj_mat, train_ds)
        # unpack based on returned tuple length
        if len(res) >= 5:
            # new API: (model, target, K0, auc, f1, _), we had some issues with this...
            _, tgt, K0, auc, _, _ = res
        else:
            # old API: (model, target, K0)
            _, tgt, K0 = res
            auc = 0.0
        aucs.append(auc)
    mean_auc = np.mean(aucs)
    if mean_auc > best_auc_o:
        best_auc_o = mean_auc
        best_K     = K
print(f" -> best K={best_K} (mean val‐AUC={best_auc_o:.3f})")


Tuning OJC…
Y = 2, train_auc = 0.625
Y = 5, train_auc = 0.625
Y = 10, train_auc = 0.625
Y = 2, train_auc = 0.357
Y = 5, train_auc = 0.357
Y = 2, train_auc = 0.651
Y = 5, train_auc = 0.651
Y = 10, train_auc = 0.651
Y = 2, train_auc = 0.438
Y = 5, train_auc = 0.438
Y = 10, train_auc = 0.438
Y = 2, train_auc = 0.631
Y = 5, train_auc = 0.631
Y = 10, train_auc = 0.631
Y = 2, train_auc = 0.433
Y = 5, train_auc = 0.433
Y = 10, train_auc = 0.433
Y = 2, train_auc = 0.938
Y = 5, train_auc = 0.938
Y = 2, train_auc = 1.000
Y = 5, train_auc = 1.000
Y = 10, train_auc = 1.000
Y = 2, train_auc = 0.375
Y = 5, train_auc = 0.375
Y = 2, train_auc = 0.417
Y = 5, train_auc = 0.417
Y = 10, train_auc = 0.417
Y = 2, train_auc = 0.567
Y = 5, train_auc = 0.567
Y = 10, train_auc = 0.567
Y = 2, train_auc = 0.688
Y = 5, train_auc = 0.688
Y = 10, train_auc = 0.688
Y = 2, train_auc = 0.551
Y = 5, train_auc = 0.551
Y = 10, train_auc = 0.551
Y = 2, train_auc = 0.628
Y = 5, train_auc = 0.628
Y = 10, train_auc = 0.628
Y

In [20]:
# pick best method (future work can also use a voting method here)
scores = {"lpsi": best_auc, "ns": best_auc_n, "ojc": best_auc_o}
method = max(scores, key=scores.get)
print(f"\nPolish → picked {method.upper()} (AUC={scores[method]:.3f})")


Polish → picked LPSI (AUC=0.652)


> We choose the method with highest mean validation AUC.  
> With more time and compute, ensembling multiple splits or adding GNN‐based plugins (e.g. GCNSI) could further improve performance, as suggested in the GraphSL article that we linked on the top.

In [21]:
# train final ensemble on each split’s train for prediction
if method == "lpsi":
    ensemble = []
    for adj_mat, train_ds, _ in splits:
        m = LPSI(); m.device = DEVICE
        with contextlib.redirect_stdout(io.StringIO()):
            m.train(adj_mat, train_ds,
                    alpha_list=[best_alpha], num_thres=10)
        ensemble.append(m)
elif method == "ns":
    ensemble = []
    for adj_mat, train_ds, _ in splits:
        m = NetSleuth(); m.device = DEVICE
        with contextlib.redirect_stdout(io.StringIO()):
            m.train(adj_mat, train_ds)
        ensemble.append(m)
else:
    ensemble = []
    for adj_mat, train_ds, _ in splits:
        m = OJC(); m.device = DEVICE
        with contextlib.redirect_stdout(io.StringIO()):
            try:
                m.train(adj_mat, train_ds, num_source=best_K)
            except TypeError:
                m.train(adj_mat, train_ds)
        ensemble.append(m)

### Prediction
For each test tree:
1. Simulate a single IC cascade.
2. Compute the graph Laplacian and infection vector.
3. Apply the chosen predictor’s `.predict(...)`
4. Pick the node with highest score as root

In [22]:
# predict on Polish test
preds = []
for _, r in test_df.iterrows():
    G0 = r["G"]
    G  = nx.convert_node_labels_to_integers(G0, first_label=0)
    n  = G.number_of_nodes()

    # simulate one cascade
    A_sp = nx.to_scipy_sparse_array(G, dtype=float, format="csr")
    out  = diffusion_generation(
        graph       = {"adj_mat": A_sp},
        diff_type   = "IC",
        sim_num     = 1,
        seed_ratio  = SEED_RATIO,
        infect_prob = INFECT_PROB
    )
    final_inf = out["diff_mat"][0][:, -1]

    # assemble Laplacian
    A = A_sp.toarray()
    L = np.diag(A.sum(axis=1)) - A
    L_t = torch.tensor(L, dtype=torch.float32, device=DEVICE)
    d_t = torch.tensor(final_inf, dtype=torch.float32, device=DEVICE)

    # get scores
    if method == "lpsi":
        scores = sum(m.predict(L_t, n, best_alpha, d_t) for m in ensemble) / len(ensemble)
    elif method == "ns":
        scores = sum(m.predict(G, best_beta, d_t) for m in ensemble) / len(ensemble)
    else:
        # OJC ensemble all have same tgt/K
        m = ensemble[0]
        scores = m.predict(G, None, best_K, d_t)

    root_pred = int(scores.argmax().item()) + 1
    preds.append({"id": r["id"], "root_pred": root_pred})

submission = pd.DataFrame(preds)
print(submission.head())

     id  root_pred
0  6436          2
1  6437          1
2  6438          1
3  6439          2
4  6440          1


### Future Opportunities...
This GraphSL‐based approach is experimental and was not submitted to Kaggle (we did not train/test on the whole data).
Next steps:
- Try out GNN‐based variants (e.g. SLVAE, GCNSI) from GraphSL
- Optimize diffusion parameters and thresholds per language
- Ensemble across multiple GraphSL algorithms

With sufficient compute and tuning, graph source localization appears to be the most best non-linear framework for root prediction, which is also stated in recent machine learning research on inverse diffusion problems. As mentioned before, this requires much more time and complexity to tune properly, but we think this could be the best direction for this problem.